In [4]:
import numpy as np
from pymatgen.core import Structure
from mendeleev import element  # For fetching covalent radius

# Function to compute adjacency matrix with manual neighbor cell looping
def compute_adjacency_matrix(poscar_file):
    #parameter for bond  decision r_1+r_2+delta
    delta=0.3

    # Determine coordinate system by reading the POSCAR file
    with open(poscar_file, 'r') as f:
        lines = f.readlines()
        # Check for 'Direct' or 'Cartesian' in the lines (case-insensitive)
        if any('direct' in line.lower() for line in lines):
            coord_type = 'direct'
        elif any('cartesian' in line.lower() for line in lines):
            coord_type = 'cartesian'
        else:
            raise ValueError("Coordinate type not found in the POSCAR file.")

    
    # Parse the POSCAR file
    structure = Structure.from_file(poscar_file)

    num_atoms = len(structure)
    atomic_numbers = [site.specie.Z for site in structure.sites]  # Atomic numbers for diagonal elements
    positions = None
    lattice = structure.lattice.matrix  # Lattice vectors (a1, a2, a3)

    if coord_type == 'direct':
        #print("Coordinates are in direct (fractional) format.")
        direct_positions = np.array([site.frac_coords for site in structure.sites])  # Direct positions
        positions = np.dot(direct_positions, lattice)  # Convert to Cartesian coordinates
    else:
        #print("Coordinates are in Cartesian format.")
        positions = np.array([site.coords for site in structure.sites])  # Atomic positions in Cartesian coordinates

    
    # Initialize the adjacency matrix
    adjacency_matrix = np.zeros((num_atoms, num_atoms))
    
    # Set diagonal elements to atomic numbers
    np.fill_diagonal(adjacency_matrix, atomic_numbers)
    
    # Get covalent radii for bonding check using mendeleev
    covalent_radii = [element(site.specie.symbol).covalent_radius/100 for site in structure.sites]
    #print(covalent_radii)
    
    # Loop over each pair of atoms in the original unit cell
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            # Loop over neighboring cells (3x3x3 grid centered on original cell)
            for k in [-1, 0, 1]:
                for l in [-1, 0, 1]:
                    for m in [-1, 0, 1]:
                        # Calculate the translation vector for the neighboring cell
                        translation = k * lattice[0] + l * lattice[1] + m * lattice[2]
                        
                        # Calculate the distance between atom i and atom j in the translated cell
                        distance_vector = positions[i] - (positions[j] + translation)
                        distance = np.linalg.norm(distance_vector)
                        
                        # Compare distance with sum of covalent radii
                        sum_covalent_radii = covalent_radii[i] + covalent_radii[j] + delta
                        
                        # If the distance is less than or equal to the sum of covalent radii, a bond exists
                        if distance <= sum_covalent_radii:
                            adjacency_matrix[i, j] += 1
                            adjacency_matrix[j, i] += 1  # Symmetric matrix

    return adjacency_matrix

# Example usage:
poscar_file = "POSCAR_Bilayers/As2+As2_4ce37585d2a3c90d+0001.vasp"
adj_matrix = compute_adjacency_matrix(poscar_file)
print(adj_matrix)


C:\Users\lorenzo.xavier\AppData\Roaming\Python\Python39\site-packages\pymatgen\core\structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


[[33.  3.  0.  0.]
 [ 3. 33.  0.  0.]
 [ 0.  0. 33.  3.]
 [ 0.  0.  3. 33.]]


In [96]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

df = pd.read_csv('output_bylayers_complete.csv',sep=";")

print(df.head)

list_ = os.listdir("POSCAR_Bilayers")
max_shape_actual = 0
array_ = []

for item in list_:
    path = Path("POSCAR_Bilayers",item)
    matrix = compute_adjacency_matrix(path)
    shape = np.shape(matrix)
    if max_shape_actual<=np.max(shape):
        max_shape_actual = np.max(shape)
    print("-"*8 + ">")
    if shape[0]==shape[1]:
        print("Square matrix")
    else:
        raise
    print("Shape:",shape)
    print("Matrix:",matrix)
    array_.append(matrix)
print("max_shape_actual:",max_shape_actual)
#array_f_ = np.hstack((array_, matrix))

<bound method NDFrame.head of                                       id  number_layers  \
0        HgTe+HgTe_5666b5bd1a1fee47+0001              2   
1      ReSe2+TaSe2_b7c464cdb704bfca+0002              2   
2      As2S3+NbTe2_601377eebb69406e+0004              2   
3        HgTe+HgTe_5666b5bd1a1fee47+0002              2   
4       Bi2Se3+Sb2_2b862700c03afb82+0001              2   
..                                   ...            ...   
435       As2+ZrS2_109ecc02574362b5+0002              2   
436     NiSe2+WTe2_109ab394c5ffa4a4+0004              2   
437    TaSe2+TiSe2_07d5e79c2248769a+0001              2   
438  Ga2Se2+Ga2Se2_276a14575112243f+0002              2   
439     As2S3+PtS2_3aff7a7a7a3a3fe4+0002              2   

                                             id_layers        formula  \
0    ['HgTe_788707fbded44ff6', 'HgTe_788707fbded44f...      HgTe+HgTe   
1    ['ReSe2_14bf5dd86ebe4186', 'TaSe2_7ec7df4e383d...    ReSe2+TaSe2   
2    ['As2S3_1b7249e6f5bc4c85', 'NbTe2_574

C:\Users\lorenzo.xavier\AppData\Roaming\Python\Python39\site-packages\pymatgen\core\structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


-------->
Square matrix
Shape: (4, 4)
Matrix: [[33.  3.  0.  0.]
 [ 3. 33.  0.  0.]
 [ 0.  0. 33.  3.]
 [ 0.  0.  3. 33.]]
-------->
Square matrix
Shape: (6, 6)
Matrix: [[33.  3.  0.  0.  0.  0.]
 [ 3. 33.  0.  0.  0.  0.]
 [ 0.  0. 31.  1.  3.  0.]
 [ 0.  0.  1. 31.  0.  3.]
 [ 0.  0.  3.  0. 16.  0.]
 [ 0.  0.  0.  3.  0. 16.]]
-------->
Square matrix
Shape: (6, 6)
Matrix: [[33.  3.  0.  0.  0.  0.]
 [ 3. 33.  0.  0.  0.  0.]
 [ 0.  0. 31.  1.  3.  0.]
 [ 0.  0.  1. 31.  0.  3.]
 [ 0.  0.  3.  0. 16.  0.]
 [ 0.  0.  0.  3.  0. 16.]]
-------->
Square matrix
Shape: (5, 5)
Matrix: [[33.  3.  0.  0.  0.]
 [ 3. 33.  0.  0.  0.]
 [ 0.  0. 42.  3.  3.]
 [ 0.  0.  3. 52.  0.]
 [ 0.  0.  3.  0. 52.]]
-------->
Square matrix
Shape: (5, 5)
Matrix: [[33.  3.  0.  0.  0.]
 [ 3. 33.  0.  0.  0.]
 [ 0.  0. 42.  3.  3.]
 [ 0.  0.  3. 52.  0.]
 [ 0.  0.  3.  0. 52.]]
-------->
Square matrix
Shape: (5, 5)
Matrix: [[33.  3.  0.  0.  0.]
 [ 3. 33.  0.  0.  0.]
 [ 0.  0. 41.  3.  3.]
 [ 0.  0.  3. 52.  0

In [9]:
import pandas as pd
from pathlib import Path

df = pd.read_csv('output_bylayers_.csv',sep=";")

columns = df.columns
print(columns)
print("len columns:",len(columns))
array_ = []
max_shape_actual = 10
x = 0
for line in df.itertuples(index=False):
    print(line.id)
    path = Path("POSCAR_Bilayers",line.id + ".vasp")
    matrix = compute_adjacency_matrix(path)
    shape = np.shape(matrix)
    if max_shape_actual<=np.max(shape):
        max_shape_actual = np.max(shape)
    print("-"*8 + ">")
    if shape[0]==shape[1]:
        print("Square matrix")
    else:
        raise
    print("Shape:",shape)
    print("Matrix:",matrix)
    print("Number:",10-np.max(shape))
    padded_matrix = np.pad(matrix, pad_width=((0,10-np.max(shape)),(0,10-np.max(shape))), mode='constant', constant_values=0)
    print("Padded Matrix:",padded_matrix)
    print("Shape:",np.shape(padded_matrix))
    dict_ = {}
    dict_["matrix"] = [str(padded_matrix).replace("\n",",")]
    array_.append(matrix)
    if x==0:
        aux_df = pd.DataFrame.from_dict(dict_,orient='index').T
        #print("aux_df:",aux_df)
        print("len line:",len(line))
        line_df = pd.DataFrame(line).T
        line_df.columns=columns
        #print("line:",line_df)
        new_df = pd.concat([line_df,aux_df],axis=1)
        print("Dict:",dict_)
        print("line_df shape:", line_df.shape)
        print("aux_df shape:", aux_df.shape)
        print("new_df shape:",new_df.shape)
        #print("new_df:",new_df) # This print error, why?
    else:
        aux_df = None
        aux_df =  pd.DataFrame.from_dict(dict_,orient='index').T
        #print("new_df_actual:",new_df_actual)
        print("len line:",len(line))
        line_df = pd.DataFrame(line).T
        line_df.columns=columns
        #print("line:",line_df)
        aux_df = pd.concat([line_df,aux_df],axis=1)
        print("Dict:",dict_)
        print("line_df shape:", line_df.shape)
        print("aux_df shape:", aux_df.shape)
        new_df = pd.concat([new_df,aux_df],axis=0)
        print("new_df shape:",new_df.shape)
        #print("new_df;",df_final) # This print error, why?
    x+=1
    print(f"Processed:{x}")
    print("-"*8 + "<")
    
print("max_shape_actual:",max_shape_actual)
print("Saving file...")
new_df.to_csv('output_bylayers_complete_2.csv',sep=';', index=False)
#print(df.head)

Index(['id', 'number_layers', 'id_layers', 'formula', 'type_ions_layers',
       'number_ions_layers', 'number_type_ions_layers', 'range_ions_layers',
       'number_ions', 'area_perc_mismatch',
       ...
       '__Z_array_max_layer1', '__atomic_radius_calculated_array_max_layer1',
       '__van_der_waals_radius_array_max_layer1',
       '__mendeleev_no_array_max_layer1',
       '__ionization_energies_array_max_layer1', '__Z_array_min_layer1',
       '__atomic_radius_calculated_array_min_layer1',
       '__van_der_waals_radius_array_min_layer1',
       '__mendeleev_no_array_min_layer1',
       '__ionization_energies_array_min_layer1'],
      dtype='object', length=131)
len columns: 131
HgTe+HgTe_5666b5bd1a1fee47+0001


C:\Users\lorenzo.xavier\AppData\Roaming\Python\Python39\site-packages\pymatgen\core\structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


-------->
Square matrix
Shape: (4, 4)
Matrix: [[80.  3.  0.  0.]
 [ 3. 52.  0.  0.]
 [ 0.  0. 80.  3.]
 [ 0.  0.  3. 52.]]
Number: 6
Padded Matrix: [[80.  3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 3. 52.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 80.  3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  3. 52.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Shape: (10, 10)
len line: 131
Dict: {'matrix': ['[[80.  3.  0.  0.  0.  0.  0.  0.  0.  0.], [ 3. 52.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0. 80.  3.  0.  0.  0.  0.  0.  0.], [ 0.  0.  3. 52.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0

C:\Users\lorenzo.xavier\AppData\Roaming\Python\Python39\site-packages\pymatgen\core\structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


-------->
Square matrix
Shape: (6, 6)
Matrix: [[75.  3.  3.  0.  0.  0.]
 [ 3. 34.  0.  0.  0.  0.]
 [ 3.  0. 34.  0.  0.  0.]
 [ 0.  0.  0. 73.  3.  3.]
 [ 0.  0.  0.  3. 34.  0.]
 [ 0.  0.  0.  3.  0. 34.]]
Number: 4
Padded Matrix: [[75.  3.  3.  0.  0.  0.  0.  0.  0.  0.]
 [ 3. 34.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 3.  0. 34.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 73.  3.  3.  0.  0.  0.  0.]
 [ 0.  0.  0.  3. 34.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0. 34.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Shape: (10, 10)
len line: 131
Dict: {'matrix': ['[[75.  3.  3.  0.  0.  0.  0.  0.  0.  0.], [ 3. 34.  0.  0.  0.  0.  0.  0.  0.  0.], [ 3.  0. 34.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0. 73.  3.  3.  0.  0.  0.  0.], [ 0.  0.  0.  3. 34.  0.  0.  0.  0.  0.], [ 0.  0.  0.  3.  0. 34.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0

...

-------->
Square matrix
Shape: (4, 4)
Matrix: [[32.  3.  0.  0.]
 [ 3. 32.  0.  0.]
 [ 0.  0. 51.  3.]
 [ 0.  0.  3. 51.]]
Number: 6
Padded Matrix: [[32.  3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 3. 32.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 51.  3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  3. 51.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Shape: (10, 10)
len line: 131
Dict: {'matrix': ['[[32.  3.  0.  0.  0.  0.  0.  0.  0.  0.], [ 3. 32.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0. 51.  3.  0.  0.  0.  0.  0.  0.], [ 0.  0.  3. 51.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0

C:\Users\lorenzo.xavier\AppData\Roaming\Python\Python39\site-packages\pymatgen\core\structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


-------->
Square matrix
Shape: (6, 6)
Matrix: [[50.  3.  3.  0.  0.  0.]
 [ 3. 34.  0.  0.  0.  0.]
 [ 3.  0. 34.  0.  0.  0.]
 [ 0.  0.  0. 50.  3.  3.]
 [ 0.  0.  0.  3. 34.  0.]
 [ 0.  0.  0.  3.  0. 34.]]
Number: 4
Padded Matrix: [[50.  3.  3.  0.  0.  0.  0.  0.  0.  0.]
 [ 3. 34.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 3.  0. 34.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 50.  3.  3.  0.  0.  0.  0.]
 [ 0.  0.  0.  3. 34.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0. 34.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Shape: (10, 10)
len line: 131
Dict: {'matrix': ['[[50.  3.  3.  0.  0.  0.  0.  0.  0.  0.], [ 3. 34.  0.  0.  0.  0.  0.  0.  0.  0.], [ 3.  0. 34.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0. 50.  3.  3.  0.  0.  0.  0.], [ 0.  0.  0.  3. 34.  0.  0.  0.  0.  0.], [ 0.  0.  0.  3.  0. 34.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0

C:\Users\lorenzo.xavier\AppData\Roaming\Python\Python39\site-packages\pymatgen\core\structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


-------->
Square matrix
Shape: (6, 6)
Matrix: [[42.  3.  3.  0.  0.  0.]
 [ 3. 52.  0.  0.  0.  0.]
 [ 3.  0. 52.  0.  0.  0.]
 [ 0.  0.  0. 42.  3.  3.]
 [ 0.  0.  0.  3. 52.  0.]
 [ 0.  0.  0.  3.  0. 52.]]
Number: 4
Padded Matrix: [[42.  3.  3.  0.  0.  0.  0.  0.  0.  0.]
 [ 3. 52.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 3.  0. 52.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 42.  3.  3.  0.  0.  0.  0.]
 [ 0.  0.  0.  3. 52.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0. 52.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Shape: (10, 10)
len line: 131
Dict: {'matrix': ['[[42.  3.  3.  0.  0.  0.  0.  0.  0.  0.], [ 3. 52.  0.  0.  0.  0.  0.  0.  0.  0.], [ 3.  0. 52.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0. 42.  3.  3.  0.  0.  0.  0.], [ 0.  0.  0.  3. 52.  0.  0.  0.  0.  0.], [ 0.  0.  0.  3.  0. 52.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0